# Предварительная обработка текста на Python

In [1]:
'''
%pip install pymorphy2
%pip install pandas
%pip install seaborn
%pip install nltk
'''

'\n%pip install pymorphy2\n%pip install pandas\n%pip install seaborn\n%pip install nltk\n'

Загружаем библиотеки

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 200
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)

#import math as mth
#import matplotlib.pyplot as plt
#from scipy import stats as st
#from statsmodels.stats.weightstats import ttest_ind
#from scipy.stats import ttest_ind_from_stats

import numpy as np
#import seaborn as sns; sns.set()
#cm = sns.light_palette("green", as_cmap=True)
#1
import nltk
import pymorphy2
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
#2
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from collections import Counter
nltk.download('stopwords')


#import os
#import io
#import re


[nltk_data] Downloading package punkt to /Users/sna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/sna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Константы

In [3]:
max_words = 1000
random_state = 42


Загружаем датасет

In [4]:
#data = pd.read_csv('/Users/sna/Dropbox/git/banks/datasets/banks.csv', sep='\t', index_col='idx', on_bad_lines='skip')



try:
    data = pd.read_csv('/Users/sna/Dropbox/git/banks/datasets/banks.csv', sep='\t', index_col='idx', on_bad_lines='skip')
except:
    data = pd.read_csv(r'C:\Users\skd23\Dropbox\git\banks\datasets\banks.csv', sep='\t', index_col='idx', on_bad_lines='skip')


'''
path = os.path.join('/' + os.sep, 'Users', 'sna', 'Dropbox', 'git', 'banks', 'datasets', 'banks.csv')
data = pd.read_csv(path, sep='\t')
'''
#data = pd.read_csv('/sna/Dropbox/git/banks/datasets/banks.csv')

#path = './datasets/banks.csv'

'''
ROOT = os.path.abspath("/Users/sna/Dropbox/git/banks/datasets/")
directories = os.listdir(ROOT)
for dir in directories:
    print(dir)
'''


'\nROOT = os.path.abspath("/Users/sna/Dropbox/git/banks/datasets/")\ndirectories = os.listdir(ROOT)\nfor dir in directories:\n    print(dir)\n'

Изучим датасет

In [5]:
list_df = (data,)
for list_df in list_df:
    print('\033[31m' + '______________info_____________________' + '\033[0m')
    display(list_df.info())
    print('\033[31m' + '______________describe_________________' + '\033[0m')
    display(list_df.describe().T)
    print('\033[31m' + '______________head_____________________' + '\033[0m')
    display(list_df.head(10))
    print('\033[31m' + '______________sample___________________' + '\033[0m')
    display(list_df.sample(10))
    print('\033[31m' + '______________tail_____________________' + '\033[0m')
    display(list_df.tail(10))
    print('\033[31m' + '_____________isna______________________' + '\033[0m')
    display(list_df.isna().sum().reset_index())
    print('\033[31m' + '_____________duplicated________________' + '\033[0m')
    display(list_df.duplicated().sum())
    print('\033[31m' + '__количество пропущенных значений по каждому из столбцов__' + '\033[0m')
    display(list_df.isnull().mean().reset_index())


______________info_____________________
<class 'pandas.core.frame.DataFrame'>
Int64Index: 13999 entries, 0 to 13998
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Score   13999 non-null  object
 1   Text    13999 non-null  object
dtypes: object(2)
memory usage: 328.1+ KB


None

______________describe_________________


,count,unique,top,freq
Score,13999,2,Positive,7000
Text,13999,13998,"В свое время коллега оставил мой телефон как контактное лицо при получении кредита.Что-то с кредитом не заладилось и теперь каждый раз, когда у него просрочка и не удается дозвониться до него — он...",2


______________head_____________________


...

,index,0
0,Score,0
1,Text,0


_____________duplicated________________


1

__количество пропущенных значений по каждому из столбцов__


,index,0
0,Score,0.0
1,Text,0.0


In [6]:
data.columns

Index(['Score', 'Text'], dtype='object')

Переведем `Text` в нижний регистр

In [7]:
data['Text'] = data['Text'].str.lower()

In [8]:
data['Text']=data['Text'].apply(str)

Определим количество негативных и позитивных оценок

In [9]:
count_score= data.groupby(['Score']).agg({'Text':'count'})\
                                    .sort_values(ascending=False, by='Score').reset_index()
count_score.columns = ['Оценка', 'Количество']
count_score



,Оценка,Количество
0,Positive,7000
1,Negative,6999


Выберем текст для токенизации

In [10]:
text = data.iloc[0]['Text']

In [11]:
text

'в альфа-банке работает замечательная девушка - ильясова орна, вежливая, отзывчивая, действительно участвует в запросе клиента, я приходила к ней подряд ровно три дня, каждый день она помнила время моего прихода, помогла оформить кредит в размере 1млн рублей, когда я пришла с партнером (передавать ей полученный кредит за покупаемый мною авто), специалист ильясова орна помогла нам вывести всю сумму в один день (а это было непросто), так что сделка состоялась и все остались довольны! моя знакомая в конце всего добавила: теперь я \xa0поняла, почему мы пришли в это отделение, к этой девушке..побольше бы таких замечательных специалистов! приобретать программу здоровье, и вообще все, что связано с альфа-банком, теперь буду только у нее!'

In [12]:
#tokens = text.astype(str).apply(word_tokenize)

In [13]:
tokens = word_tokenize(text)


In [14]:
tokens[:10]

['в',
 'альфа-банке',
 'работает',
 'замечательная',
 'девушка',
 '-',
 'ильясова',
 'орна',
 ',',
 'вежливая']

Удалим знаки препинания

In [15]:
#punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...']
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...', '«', '»', ';', '-', '--']


In [16]:
only_words = []
for token in tokens:
    if token not in punctuation_marks:
        only_words.append(token)

In [17]:
only_words[:10]

['в',
 'альфа-банке',
 'работает',
 'замечательная',
 'девушка',
 'ильясова',
 'орна',
 'вежливая',
 'отзывчивая',
 'действительно']

Приводим слова к нормальной форме

In [18]:
morph = pymorphy2.MorphAnalyzer()

In [19]:
lemmas = []
for token in only_words:
    lemmas.append(morph.parse(token)[0].normal_form)

In [20]:
lemmas[:10]

['в',
 'альфа-банк',
 'работать',
 'замечательный',
 'девушка',
 'ильясова',
 'орный',
 'вежливый',
 'отзывчивый',
 'действительно']

удалим стоп-слова

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/sna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
stop_words = stopwords.words('russian')

In [23]:
stop_words[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

Функция предварительной обработки

In [24]:
def preprocess(text, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return preprocessed_text

In [25]:
#punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...']
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...', '«', '»', ';', '-', '--']
stop_words = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

In [26]:
#data[0:10].apply(lambda row: preprocess(row['Text'], punctuation_marks, stop_words, morph), axis=1)

In [27]:
data['Preprocessed_texts'] = data.apply(lambda row: preprocess(row['Text'], punctuation_marks, stop_words, morph), axis=1)

In [28]:
data.head()

,Score,Text,Preprocessed_texts
idx,,,
0,Positive,"в альфа-банке работает замечательная девушка - ильясова орна, вежливая, отзывчивая, действительно участвует в запросе клиента, я приходила к ней подряд ровно три дня, каждый день она помнила время...","[альфа-банк, работать, замечательный, девушка, ильясова, орный, вежливый, отзывчивый, действительно, участвовать, запрос, клиент, приходить, подряд, ровно, день, каждый, день, помнить, время, мой,..."
1,Negative,"оформляя рассрочку в м. видео в меге тёплый стан. попросила менеджера, чтобы заявка была без страховки. в итоге, когда подписывала договор, в договоре оказались лишние почти 6 тысяч, когда менедже...","[оформлять, рассрочка, м., видео, мег, тёплый, стан, попросить, менеджер, заявка, страховка, итог, подписывать, договор, договор, оказаться, лишний, 6, тысяча, менеджер, стать, спрашивать, это, 6,..."
2,Positive,"очень порадовала оперативность работы в банке. заказал через интернет карту visa classic, потребовалось подойти в банк для подписания заявления. в тот же день мне открыли текущий счет с доступом ч...","[очень, порадовать, оперативность, работа, банк, заказать, интернет, карта, visa, classic, потребоваться, подойти, банк, подписание, заявление, день, открыть, текущий, счёт, доступ, интернет, услу..."
3,Negative,"имела неосторожность оформить потреб. кредит в альфа-банке - очень жалею. во-первых, карту при оформлении кредита мне не выдали (объяснили тем, что их сейчас нет в наличии). во-вторых, никакую инф...","[иметь, неосторожность, оформить, потреба, кредит, альфа-банк, очень, жалеть, во-первых, карта, оформление, кредит, выдать, объяснить, наличие, во-вторых, никакой, информация, телефон, предоставля..."
4,Negative,"небольшая предыстория: нашел на сайте mdm банка интересную ко-бренд карту «техносила – mdm банк», подал заявку на нее, и скоро позвонили с горячей линии. предложили кредитную карту, на что я в при...","[небольшой, предыстория, найти, сайт, mdm, банк, интересный, ко-бренд, карта, техносить, –, mdm, банк, подать, заявка, скоро, позвонить, горячий, линия, предложить, кредитный, карта, принцип, отве..."


In [29]:
data.columns

Index(['Score', 'Text', 'Preprocessed_texts'], dtype='object')

In [30]:
#1
#prep_text = data['Preprocessed_texts']
#prep_text.to_frame()

In [31]:
#1
#clean_df = data[data['Preprocessed_texts'].str.len() >= 100]

In [32]:
#1
#clean_df.Preprocessed_texts.to_frame()

Определим тональность текста с помощью мешка слов

Считаем частоту слов во всех отзывых

In [33]:
words = Counter()

In [34]:
for txt in data['Preprocessed_texts']:
    words.update(txt)


Создаем словарь, упорядоченный по частоте

В словаре будем использовать 2 специальных кода:

 - Код заполнитель: 0
 - Неизвестное слово: 1

Нумерация слов в словаре начинается с 2.

In [35]:
# Словарь, отображающий слова в коды
word_to_index = dict()
# Словарь, отображающий коды в слова
index_to_word = dict()

Создаем словари

In [36]:
for i, word in enumerate(words.most_common(max_words - 2)):
    word_to_index[word[0]] = i + 2
    index_to_word[i + 2] = word[0]

Функция для преобразования списка слов в список кодов

In [37]:
def text_to_sequence(txt, word_to_index):
    seq = []
    for word in txt:
        index = word_to_index.get(word, 1) # 1 означает неизвестное слово
        # Неизвестные слова не добавляем в выходную последовательность
        if index != 1:
            seq.append(index)
    return seq

Преобразуем все тексты в последовательность кодов слов

In [38]:
data['Sequences'] = data.apply(lambda row: text_to_sequence(row['Preprocessed_texts'], word_to_index), axis=1)

In [39]:
data.head()

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
0,Positive,"в альфа-банке работает замечательная девушка - ильясова орна, вежливая, отзывчивая, действительно участвует в запросе клиента, я приходила к ней подряд ровно три дня, каждый день она помнила время...","[альфа-банк, работать, замечательный, девушка, ильясова, орный, вежливый, отзывчивый, действительно, участвовать, запрос, клиент, приходить, подряд, ровно, день, каждый, день, помнить, время, мой,...","[62, 43, 917, 78, 205, 358, 410, 13, 155, 5, 147, 5, 576, 19, 6, 162, 72, 9, 299, 54, 46, 23, 9, 954, 90, 101, 162, 186, 20, 5, 4, 271, 219, 300, 491, 214, 259, 103, 362, 4, 12, 78, 242, 25, 917, ..."
1,Negative,"оформляя рассрочку в м. видео в меге тёплый стан. попросила менеджера, чтобы заявка была без страховки. в итоге, когда подписывала договор, в договоре оказались лишние почти 6 тысяч, когда менедже...","[оформлять, рассрочка, м., видео, мег, тёплый, стан, попросить, менеджер, заявка, страховка, итог, подписывать, договор, договор, оказаться, лишний, 6, тысяча, менеджер, стать, спрашивать, это, 6,...","[268, 856, 172, 84, 63, 156, 136, 785, 28, 28, 146, 668, 461, 406, 84, 94, 427, 4, 461, 406, 94, 59, 160, 107, 802, 34, 424, 9, 665, 461, 406, 280, 276, 28, 240, 156, 285, 276, 386, 156, 136, 278,..."
2,Positive,"очень порадовала оперативность работы в банке. заказал через интернет карту visa classic, потребовалось подойти в банк для подписания заявления. в тот же день мне открыли текущий счет с доступом ч...","[очень, порадовать, оперативность, работа, банк, заказать, интернет, карта, visa, classic, потребоваться, подойти, банк, подписание, заявление, день, открыть, текущий, счёт, доступ, интернет, услу...","[18, 911, 29, 2, 476, 184, 3, 629, 417, 2, 862, 27, 5, 119, 416, 11, 793, 184, 44, 895, 107, 104, 73, 23, 3, 144, 537, 290, 3, 386, 46, 694, 137, 56, 121, 18, 92, 12, 146, 18, 64, 53, 8, 366, 44, ..."
3,Negative,"имела неосторожность оформить потреб. кредит в альфа-банке - очень жалею. во-первых, карту при оформлении кредита мне не выдали (объяснили тем, что их сейчас нет в наличии). во-вторых, никакую инф...","[иметь, неосторожность, оформить, потреба, кредит, альфа-банк, очень, жалеть, во-первых, карта, оформление, кредит, выдать, объяснить, наличие, во-вторых, никакой, информация, телефон, предоставля...","[118, 72, 9, 62, 18, 3, 168, 9, 137, 157, 652, 81, 70, 32, 812, 3, 139, 831, 2, 15, 140, 106, 441, 12, 2, 55, 395, 9, 141, 2, 313, 425, 40, 13, 126, 281, 9, 428, 126, 4, 2, 118, 61, 369, 268, 9, 8..."
4,Negative,"небольшая предыстория: нашел на сайте mdm банка интересную ко-бренд карту «техносила – mdm банк», подал заявку на нее, и скоро позвонили с горячей линии. предложили кредитную карту, на что я в при...","[небольшой, предыстория, найти, сайт, mdm, банк, интересный, ко-бренд, карта, техносить, –, mdm, банк, подать, заявка, скоро, позвонить, горячий, линия, предложить, кредитный, карта, принцип, отве...","[419, 276, 87, 2, 649, 3, 123, 2, 297, 63, 33, 165, 133, 131, 21, 3, 771, 110, 152, 216, 97, 220, 21, 451, 130, 21, 283, 15, 74, 12, 2, 390, 216, 3, 253, 92, 390, 2, 562, 185, 482, 192, 384, 12, 1..."


Готовим данные для обучения

Преобразуем текстовые метрики классов в числовые

In [40]:
mapping = {'Negative': 0, 'Positive': 1}

In [41]:
data.replace({'Score': mapping}, inplace=True)

In [42]:
data.head()

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
0,1,"в альфа-банке работает замечательная девушка - ильясова орна, вежливая, отзывчивая, действительно участвует в запросе клиента, я приходила к ней подряд ровно три дня, каждый день она помнила время...","[альфа-банк, работать, замечательный, девушка, ильясова, орный, вежливый, отзывчивый, действительно, участвовать, запрос, клиент, приходить, подряд, ровно, день, каждый, день, помнить, время, мой,...","[62, 43, 917, 78, 205, 358, 410, 13, 155, 5, 147, 5, 576, 19, 6, 162, 72, 9, 299, 54, 46, 23, 9, 954, 90, 101, 162, 186, 20, 5, 4, 271, 219, 300, 491, 214, 259, 103, 362, 4, 12, 78, 242, 25, 917, ..."
1,0,"оформляя рассрочку в м. видео в меге тёплый стан. попросила менеджера, чтобы заявка была без страховки. в итоге, когда подписывала договор, в договоре оказались лишние почти 6 тысяч, когда менедже...","[оформлять, рассрочка, м., видео, мег, тёплый, стан, попросить, менеджер, заявка, страховка, итог, подписывать, договор, договор, оказаться, лишний, 6, тысяча, менеджер, стать, спрашивать, это, 6,...","[268, 856, 172, 84, 63, 156, 136, 785, 28, 28, 146, 668, 461, 406, 84, 94, 427, 4, 461, 406, 94, 59, 160, 107, 802, 34, 424, 9, 665, 461, 406, 280, 276, 28, 240, 156, 285, 276, 386, 156, 136, 278,..."
2,1,"очень порадовала оперативность работы в банке. заказал через интернет карту visa classic, потребовалось подойти в банк для подписания заявления. в тот же день мне открыли текущий счет с доступом ч...","[очень, порадовать, оперативность, работа, банк, заказать, интернет, карта, visa, classic, потребоваться, подойти, банк, подписание, заявление, день, открыть, текущий, счёт, доступ, интернет, услу...","[18, 911, 29, 2, 476, 184, 3, 629, 417, 2, 862, 27, 5, 119, 416, 11, 793, 184, 44, 895, 107, 104, 73, 23, 3, 144, 537, 290, 3, 386, 46, 694, 137, 56, 121, 18, 92, 12, 146, 18, 64, 53, 8, 366, 44, ..."
3,0,"имела неосторожность оформить потреб. кредит в альфа-банке - очень жалею. во-первых, карту при оформлении кредита мне не выдали (объяснили тем, что их сейчас нет в наличии). во-вторых, никакую инф...","[иметь, неосторожность, оформить, потреба, кредит, альфа-банк, очень, жалеть, во-первых, карта, оформление, кредит, выдать, объяснить, наличие, во-вторых, никакой, информация, телефон, предоставля...","[118, 72, 9, 62, 18, 3, 168, 9, 137, 157, 652, 81, 70, 32, 812, 3, 139, 831, 2, 15, 140, 106, 441, 12, 2, 55, 395, 9, 141, 2, 313, 425, 40, 13, 126, 281, 9, 428, 126, 4, 2, 118, 61, 369, 268, 9, 8..."
4,0,"небольшая предыстория: нашел на сайте mdm банка интересную ко-бренд карту «техносила – mdm банк», подал заявку на нее, и скоро позвонили с горячей линии. предложили кредитную карту, на что я в при...","[небольшой, предыстория, найти, сайт, mdm, банк, интересный, ко-бренд, карта, техносить, –, mdm, банк, подать, заявка, скоро, позвонить, горячий, линия, предложить, кредитный, карта, принцип, отве...","[419, 276, 87, 2, 649, 3, 123, 2, 297, 63, 33, 165, 133, 131, 21, 3, 771, 110, 152, 216, 97, 220, 21, 451, 130, 21, 283, 15, 74, 12, 2, 390, 216, 3, 253, 92, 390, 2, 562, 185, 482, 192, 384, 12, 1..."


Выделяем данные для обучения и тестирования

In [43]:
train, test = train_test_split(data, test_size=0.2)

In [44]:
#80% данных для обучения
train.head()

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
5261,0,в полном расчете кредита на первичный рынок недвижимости не значились расходы на оценку квартиры (приложение к кредитному договору). в предварительном договоре о залоге (ипотеке) стороны договорил...,"[полный, расчёт, кредит, первичный, рынок, недвижимость, значиться, расход, оценка, квартира, приложение, кредитный, договор, предварительный, договор, залог, ипотека, сторона, договориться, оцено...","[230, 814, 9, 400, 324, 764, 21, 28, 28, 454, 250, 723, 324, 299, 723, 474, 89, 596, 927, 2, 403, 400, 53, 391, 342, 820, 283, 484, 103, 91, 286, 796, 2, 222, 115, 5, 61, 99, 37, 214, 486, 548, 38..."
765,1,"вот собственно и я решил выразить мнение о банке. в жизненной суете чуть не забыл про него. итак, потребительский кредит оформлялся в санкт-петербурге в салоне сотовой связи, после отказа в хоумкр...","[собственно, решить, выразить, мнение, банк, жизненный, суета, забыть, итак, потребительский, кредит, оформляться, санкт-петербург, салон, сотовый, связь, отказ, хоумкредит, прийтись, обратиться, ...","[896, 38, 473, 732, 2, 494, 447, 9, 968, 213, 248, 164, 74, 698, 539, 214, 197, 5, 140, 67, 63, 9, 68, 484, 975, 85, 46, 248, 157, 425, 58, 669, 238, 770, 83, 463, 269, 702, 152, 70, 547, 68, 354,..."
4,0,"небольшая предыстория: нашел на сайте mdm банка интересную ко-бренд карту «техносила – mdm банк», подал заявку на нее, и скоро позвонили с горячей линии. предложили кредитную карту, на что я в при...","[небольшой, предыстория, найти, сайт, mdm, банк, интересный, ко-бренд, карта, техносить, –, mdm, банк, подать, заявка, скоро, позвонить, горячий, линия, предложить, кредитный, карта, принцип, отве...","[419, 276, 87, 2, 649, 3, 123, 2, 297, 63, 33, 165, 133, 131, 21, 3, 771, 110, 152, 216, 97, 220, 21, 451, 130, 21, 283, 15, 74, 12, 2, 390, 216, 3, 253, 92, 390, 2, 562, 185, 482, 192, 384, 12, 1..."
2405,0,"вчера вечером пыталась внести очередной платеж по ипотечному кредиту. пришла в отделение на курской в 19.20. передо мной в очереди оказалось 13 (!) человек. при этом в зале сидело 3 сотрудника, но...","[вчера, вечером, пытаться, внести, очередной, платёж, ипотечный, кредит, прийти, отделение, курский, 19.20., перед, я, очередь, оказаться, 13, человек, зал, сидеть, 3, сотрудник, лишь, 1, обслужив...","[524, 597, 280, 187, 338, 30, 548, 9, 46, 12, 90, 66, 146, 55, 962, 549, 122, 7, 530, 176, 625, 13, 508, 264, 56, 172, 383, 294, 497, 774, 87, 757, 383, 709, 45, 2, 408, 51, 93, 87, 2, 4, 57, 66, ..."
7163,0,"много лет я являюсь вкладчиком мдм-банка, всегда была очень довольна обслуживанием. сегодня, 29.12.2016г. у меня закончился срок вклада.«великолепная семерка» в долларах сша и я обратилась в отдел...","[год, являться, вкладчик, мдм-банк, очень, довольный, обслуживание, сегодня, 29.12.2016г, закончиться, срок, вклада., великолепный, семёрка, доллар, сша, обратиться, отделение, правопреемник, мдм-...","[14, 86, 934, 18, 300, 40, 109, 617, 65, 618, 74, 12, 123, 246, 178, 127, 198, 347, 273, 316, 455, 10, 350, 455, 119, 47, 71, 68, 450, 287, 174, 711, 389, 66, 298, 7, 2, 98, 47, 333, 304, 119, 54,..."


In [45]:
#данные для тестирования 20%
test.head()

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
8973,1,"обслуживаюсь в офисе шуваловский в москве.стоит отметить, что я являюсь требовательным клиентом. не пользуюсь банковскими продуктами если не понимаю как они работают и без колебаний расстаюсь с ба...","[обслуживаться, офис, шуваловский, москве.стоять, отметить, являться, требовательный, клиент, пользоваться, банковский, продукт, понимать, работать, колебание, расставаться, банк, считать, плохо, ...","[363, 24, 321, 86, 13, 77, 223, 243, 232, 43, 2, 260, 17, 2, 210, 332, 22, 677, 115, 24, 448, 29, 13, 4, 397, 40, 71, 243, 929, 7, 24, 232, 399, 13, 328, 249, 7, 8, 436, 17, 29, 140, 124, 18, 480,..."
13379,0,"наша компания является корпоративным клиентом альфа банка, следовательно и зарплатный проект у нас там же. к сожалению, сотрудники не могут выбрать банк для себя. у нас сотрудники постоянно жалова...","[наш, компания, являться, корпоративный, клиент, альфа, банк, следовательно, зарплатный, проект, сожаление, сотрудник, мочь, выбрать, банк, сотрудник, постоянно, жаловаться, альфа, банк, верить, п...","[134, 342, 86, 13, 167, 2, 344, 990, 394, 7, 22, 384, 2, 7, 506, 167, 2, 154, 875, 77, 31, 167, 2, 3, 266, 31, 3, 567, 14, 5, 38, 3, 15, 2, 408, 88, 6, 3, 364, 158, 597, 560, 31, 167, 2, 3, 829, 1..."
6634,0,добрый день. с 2006 года являюсь клиентом альфа банка. до настоящего времени была абсолютна довольна и банком и обслуживанием. но январе 2015 года столкнулась с ужасным обслуживанием в отделении г...,"[добрый, день, 2006, год, являться, клиент, альфа, банк, настоящий, время, абсолютный, довольный, банк, обслуживание, январь, 2015, год, столкнуться, ужасный, обслуживание, отделение, грибоедовски...","[182, 5, 14, 86, 13, 167, 2, 790, 19, 300, 2, 40, 435, 718, 14, 875, 40, 12, 178, 968, 414, 809, 435, 718, 14, 74, 79, 12, 541, 9, 213, 915, 319, 636, 14, 7, 12, 488, 576, 143, 35, 9, 91, 9, 15, 9..."
4130,0,"я ещё не встречала такого некомпетентного банка как ваш. грош цена всем вашим обещаниям и извинениям. больше месяца назад (08.12.14) вы принесли извинения и пообещали, что больше мой номер телефон...","[ещё, встречать, такой, некомпетентный, банк, ваш, грош, цена, весь, ваш, обещание, извинение, месяц, назад, 08.12.14, принести, извинение, пообещать, номер, телефон, ваш, база, данные, фигурирова...","[231, 974, 25, 2, 42, 64, 42, 834, 34, 211, 852, 834, 50, 32, 42, 650, 286, 325, 109, 195, 17, 50, 42, 2, 78, 124, 49, 317, 4, 502, 295, 622, 383, 48, 376, 186, 741, 426]"
10192,0,"22.11.2016 обратилась в отделение бинбанка, чтобы сделать карточку для получения з/п. сотрудник г-т а. а. сообщил, что через 2-3 дня должны позвонить и сообщить о том, что карточка готова. но чере...","[22.11.2016, обратиться, отделение, бинбанка, сделать, карточка, получение, з/п, сотрудник, г-т, а., а., сообщить, 2-3, день, должный, позвонить, сообщить, карточка, готовый, 1,5, неделя, никто, п...","[74, 12, 246, 57, 227, 161, 7, 98, 5, 73, 33, 98, 227, 291, 104, 163, 33, 2, 227, 146, 291, 161, 26, 365, 146, 7, 722, 6, 36, 5, 6, 117, 53, 115, 5, 253, 38, 26, 564, 12, 515, 78, 8, 869, 90, 6, 1..."


Разделяем метки классов и данные для обучения

Данные для обучения

In [46]:
x_train_seq = train['Sequences']#данные для обучения
y_train = train['Score']#метки(ответы)

In [47]:
x_train_seq.to_frame().head()#данные для обучения

,Sequences
idx,
5261,"[230, 814, 9, 400, 324, 764, 21, 28, 28, 454, 250, 723, 324, 299, 723, 474, 89, 596, 927, 2, 403, 400, 53, 391, 342, 820, 283, 484, 103, 91, 286, 796, 2, 222, 115, 5, 61, 99, 37, 214, 486, 548, 38..."
765,"[896, 38, 473, 732, 2, 494, 447, 9, 968, 213, 248, 164, 74, 698, 539, 214, 197, 5, 140, 67, 63, 9, 68, 484, 975, 85, 46, 248, 157, 425, 58, 669, 238, 770, 83, 463, 269, 702, 152, 70, 547, 68, 354,..."
4,"[419, 276, 87, 2, 649, 3, 123, 2, 297, 63, 33, 165, 133, 131, 21, 3, 771, 110, 152, 216, 97, 220, 21, 451, 130, 21, 283, 15, 74, 12, 2, 390, 216, 3, 253, 92, 390, 2, 562, 185, 482, 192, 384, 12, 1..."
2405,"[524, 597, 280, 187, 338, 30, 548, 9, 46, 12, 90, 66, 146, 55, 962, 549, 122, 7, 530, 176, 625, 13, 508, 264, 56, 172, 383, 294, 497, 774, 87, 757, 383, 709, 45, 2, 408, 51, 93, 87, 2, 4, 57, 66, ..."
7163,"[14, 86, 934, 18, 300, 40, 109, 617, 65, 618, 74, 12, 123, 246, 178, 127, 198, 347, 273, 316, 455, 10, 350, 455, 119, 47, 71, 68, 450, 287, 174, 711, 389, 66, 298, 7, 2, 98, 47, 333, 304, 119, 54,..."


In [48]:
y_train.to_frame().head(10)

,Score
idx,
5261,0
765,1
4,0
2405,0
7163,0
9877,1
2533,1
7360,0
3377,1


Данные для тестирования

In [49]:
x_test_seq = test['Sequences']#данные для обучения
y_test = test['Score']#метки(ответы)

In [50]:
x_test_seq.to_frame().head()#данные для обучения

,Sequences
idx,
8973,"[363, 24, 321, 86, 13, 77, 223, 243, 232, 43, 2, 260, 17, 2, 210, 332, 22, 677, 115, 24, 448, 29, 13, 4, 397, 40, 71, 243, 929, 7, 24, 232, 399, 13, 328, 249, 7, 8, 436, 17, 29, 140, 124, 18, 480,..."
13379,"[134, 342, 86, 13, 167, 2, 344, 990, 394, 7, 22, 384, 2, 7, 506, 167, 2, 154, 875, 77, 31, 167, 2, 3, 266, 31, 3, 567, 14, 5, 38, 3, 15, 2, 408, 88, 6, 3, 364, 158, 597, 560, 31, 167, 2, 3, 829, 1..."
6634,"[182, 5, 14, 86, 13, 167, 2, 790, 19, 300, 2, 40, 435, 718, 14, 875, 40, 12, 178, 968, 414, 809, 435, 718, 14, 74, 79, 12, 541, 9, 213, 915, 319, 636, 14, 7, 12, 488, 576, 143, 35, 9, 91, 9, 15, 9..."
4130,"[231, 974, 25, 2, 42, 64, 42, 834, 34, 211, 852, 834, 50, 32, 42, 650, 286, 325, 109, 195, 17, 50, 42, 2, 78, 124, 49, 317, 4, 502, 295, 622, 383, 48, 376, 186, 741, 426]"
10192,"[74, 12, 246, 57, 227, 161, 7, 98, 5, 73, 33, 98, 227, 291, 104, 163, 33, 2, 227, 146, 291, 161, 26, 365, 146, 7, 722, 6, 36, 5, 6, 117, 53, 115, 5, 253, 38, 26, 564, 12, 515, 78, 8, 869, 90, 6, 1..."


In [51]:
y_test.to_frame().head(10)

,Score
idx,
8973,1
13379,0
6634,0
4130,0
10192,0
8783,0
1385,0
7265,0
13780,1


Создаем мешок слов

In [52]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for index in sequence:
            results[i, index] += 1.
    return results

In [53]:
x_train = vectorize_sequences(x_train_seq, max_words)

In [54]:
x_test = vectorize_sequences(x_test_seq, max_words)

In [55]:
x_train[0][:100]

array([ 0.,  0., 13.,  0.,  1.,  2.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,
        0.,  1.,  1.,  2.,  1.,  0.,  1.,  0.,  4.,  0.,  0.,  0.,  0.,
        0.,  0.,  7.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  2.,  4.,  0.,
        3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        1.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  4.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.])

In [56]:
len(x_train[0])

1000

Создаем модель машинного обучения

In [57]:
lr = LogisticRegression(random_state=random_state, max_iter=500)

Обучаем модель машинного обучения

In [58]:
lr.fit(x_train, y_train)

LogisticRegression(max_iter=500, random_state=42)

Оцениваем качество обучения на тестовом наборе данных

Определяем долю правильных ответов (accuracy) на тестовом наборе данных

In [59]:
lr.score(x_test, y_test)

0.9228571428571428

Применяем модель для определения тональности отзыва на банк

Позитивный отзыв

In [60]:
positive_text = """Брал кредит в Мегабанке на автомобиль. Выдали за один день. Никаких скрытых комиссий и переплат. 
У банка удобное мобильное приложение, через которое можно быстро отправить ежемесячный платеж. 
Досрочное гасить начал через три месяца. Я доволен оперативностью и удобством. Огромное спасибо!
"""

Подготовка текста к обработке

In [61]:
positive_preprocessed_text = preprocess(positive_text, stop_words, punctuation_marks, morph)

In [62]:
positive_preprocessed_text

['брать',
 'кредит',
 'мегабанк',
 'автомобиль',
 'выдать',
 'день',
 'никакой',
 'скрытый',
 'комиссия',
 'переплата',
 'банк',
 'удобный',
 'мобильный',
 'приложение',
 'который',
 'быстро',
 'отправить',
 'ежемесячный',
 'платёж',
 'досрочный',
 'гасить',
 'начать',
 'месяц',
 'довольный',
 'оперативность',
 'удобство',
 'огромный',
 'спасибо']

In [63]:
positive_seq = text_to_sequence(positive_preprocessed_text, word_to_index)

In [64]:
positive_seq

[171,
 9,
 964,
 137,
 5,
 81,
 80,
 2,
 288,
 306,
 764,
 8,
 129,
 206,
 490,
 30,
 359,
 279,
 34,
 300,
 911,
 313,
 75]

In [65]:
positive_bow = vectorize_sequences([positive_seq], max_words)

In [66]:
positive_bow[0][0:100]

array([0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Выполняем распознавание

In [67]:
result = lr.predict(positive_bow)

In [68]:
result

array([1])

Негативный отзыв

In [69]:
negative_text = """Взял кредит в ТакСебеБанке на автомобиль. В договор включили обязательный контракт
на помощь на дороге, который мне не нужен. Узнал об этом только во время подписания договора, иначе бы отказался.
Альтернативы была страхование жизни, но мне это даже не предложили. Скорее всего, менеджер продвигает
продажи услуг этой компании в ущерб интересов клиента. Как минимум, непорядочно и непрофессионально.
У банка ужасное мобильное приложение, из-за которого с меня взяли штраф 10 тыс.руб. По требованиям 
банка после покупки автомобиля в приложении нужно загрузить ПТС. Я загрузил и проверил, что ПТС в приложении есть.
Но через некоторое время ПТС из приложения пропал и с меня взяли штраф. Никому не рекомендую связываться с ТакСебеБанком.
"""

In [70]:
negative_preprocessed_text = preprocess(negative_text, stop_words, punctuation_marks, morph)
negative_seq = text_to_sequence(negative_preprocessed_text, word_to_index)
negative_bow = vectorize_sequences([negative_seq], max_words)

In [71]:
result = lr.predict(negative_bow)

In [72]:
result

array([1])

In [73]:
result = lr.predict_proba(negative_bow)

In [74]:
result

array([[0.17866721, 0.82133279]])